<a href="https://colab.research.google.com/github/NickRios1/de-zoomcamp/blob/main/workshops/dlt/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator_sum = 0
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)
    generator_sum += sqrt_value

print("Answer to Question #1:",generator_sum)

#Question 2
target = 13
generator = square_root_generator(target)
answer = 0
for sqrt_value in generator:
  answer = sqrt_value

print("Answer to Question #2:",answer)



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
Answer to Question #1: 8.382332347441762
Answer to Question #2: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [7]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [11]:
import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

# Create Data Pipeline
pipeline = dlt.pipeline(destination='duckdb', dataset_name='population_details')

#Load Data of person_1 generator
info = pipeline.run(people_1,
										table_name="population_1",
										write_disposition="replace",
                    primary_key="ID")

print(info)

# show the table generated from above
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n Population Data Loaded from Generator-1")
people = conn.sql("SELECT * FROM population_1").df()
display(people)

print("\n\n\n Sum of All Ages from Current Data")
total_age_1 = conn.sql("SELECT SUM(Age) FROM population_1").df()
display(total_age_1)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset population_details
The duckdb destination used duckdb:////content/dlt_col

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ population_1        │
│ population_2        │
└─────────────────────┘




 Population Data Loaded from Generator-1


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707595981.6398842,as7h2qc3k9nxKA,None
1,2,Person_2,27,City_A,1707595981.6398842,N65bZRyWmg3lKw,None
2,3,Person_3,28,City_A,1707595981.6398842,NZiN27aP07Wxtg,None
3,4,Person_4,29,City_A,1707595981.6398842,FC5ZAL7f1+8/ag,None
4,5,Person_5,30,City_A,1707595981.6398842,Q3qMa1PRjcttBA,None





 Sum of All Ages from Current Data


,sum(Age)
0,140.0


In [9]:
info_2 = pipeline.run(people_2,
										table_name="population_1",
										write_disposition="append",
                    primary_key="ID")

print(info_2)

print("\n\n\n Population Data Loaded from Generator-2")
people = conn.sql("SELECT * FROM population_1").df()
display(people)

print("\n\n\n Sum of All Ages from Updated Data")
total_age_2 = conn.sql("SELECT SUM(Age) FROM population_1").df()
display(total_age_2)


Pipeline dlt_colab_kernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset population_details
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707595094.870523 is LOADED and contains no failed jobs



 Population Data Loaded from Generator-2


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707594826.8282096,FH9sNa9/VpRTsA,None
1,2,Person_2,27,City_A,1707594826.8282096,D8qAlDGE/IqYSw,None
2,3,Person_3,28,City_A,1707594826.8282096,SvGhFB78SyVKzQ,None
3,4,Person_4,29,City_A,1707594826.8282096,bqLGHAT70SHzEQ,None
4,5,Person_5,30,City_A,1707594826.8282096,ZiEE8eHke+ZdpQ,None
5,3,Person_3,33,City_B,1707595094.870523,5gEuy/jdbinAXA,Job_3
6,4,Person_4,34,City_B,1707595094.870523,WiCzRDbrTX7mYw,Job_4
7,5,Person_5,35,City_B,1707595094.870523,CT9KnO6AN49M5g,Job_5
8,6,Person_6,36,City_B,1707595094.870523,EgRqYiPQY4TkAw,Job_6
9,7,Person_7,37,City_B,1707595094.870523,+jEiOAwujUQ4ag,Job_7





 Sum of All Ages from Updated Data


,sum(Age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [10]:

# Create Data Pipeline
new_pipeline = dlt.pipeline(destination='duckdb', dataset_name='population_details')

#Load Data of person_1 generator
new_info = pipeline.run(people_1,
										table_name="population_2",
										write_disposition="replace",
                    primary_key="ID")

print(new_info)

# show the table generated from above
conn_2 = duckdb.connect(f"{new_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn_2.sql(f"SET search_path = '{new_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn_2.sql("show tables"))


print("\n\n\n Population Data Loaded from Generator-1")
people_replace = conn_2.sql("SELECT * FROM population_2").df()
display(people_replace)

print("\n\n\n Sum of All Ages from Current Data")
total_age_replace = conn.sql("SELECT SUM(Age) FROM population_2").df()
display(total_age_replace)

new_info_2 = pipeline.run(people_2,
										table_name="population_2",
										write_disposition="merge",
                    primary_key="ID")

print(new_info_2)

print("\n\n\n Population Data Loaded from Generator-2")
people_merge = conn_2.sql("SELECT * FROM population_2").df()
display(people_merge)

print("\n\n\n Sum of All Ages from Updated Data")
total_age_merge = conn_2.sql("SELECT SUM(Age) FROM population_2").df()
display(total_age_merge)




Pipeline dlt_colab_kernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset population_details
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707595907.7892764 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ population_1        │
│ population_2        │
└─────────────────────┘




 Population Data Loaded from Generator-1


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707595907.7892764,pcQYRrMmkaRKwg
1,2,Person_2,27,City_A,1707595907.7892764,bzg7if0/JLFWNA
2,3,Person_3,28,City_A,1707595907.7892764,EpnhH573P9PkwA
3,4,Person_4,29,City_A,1707595907.7892764,Q4P9v+quYOrBHQ
4,5,Person_5,30,City_A,1707595907.7892764,vviw0694KyaBWw





 Sum of All Ages from Current Data


,sum(Age)
0,140.0


Pipeline dlt_colab_kernel_launcher load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset population_details
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707595908.5071406 is LOADED and contains no failed jobs



 Population Data Loaded from Generator-2


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707595907.7892764,pcQYRrMmkaRKwg,None
1,2,Person_2,27,City_A,1707595907.7892764,bzg7if0/JLFWNA,None
2,5,Person_5,35,City_B,1707595908.5071406,E+JvLv6DRYyajw,Job_5
3,4,Person_4,34,City_B,1707595908.5071406,2XLGtR9ybfEfIg,Job_4
4,8,Person_8,38,City_B,1707595908.5071406,g0Z/Cl5I91KcHQ,Job_8
5,7,Person_7,37,City_B,1707595908.5071406,WkeXHWZ1WhAtKg,Job_7
6,3,Person_3,33,City_B,1707595908.5071406,gKEn6KLzYkLpzw,Job_3
7,6,Person_6,36,City_B,1707595908.5071406,Z2c7NASUT7C7LQ,Job_6





 Sum of All Ages from Updated Data


,sum(Age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX